In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import Dropout, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [33]:
#  This function finds the percentage of each value compared to the previous one
#  For the first value, since there is not a previous one, I find the difference between the first element and the second one
def AvgPrevious(nparray):
    finalarray = []

    for i in range(len(nparray)):

        if i > 0:
            finalarray.append(nparray[i] - nparray[i - 1]/nparray[i])
        else:
            finalarray.append(nparray[i] - nparray[i + 1]/nparray[i])
    return finalarray

In [3]:
rawdata = pd.read_csv('data/bitcoin.csv',usecols=[3,8])
data = rawdata.as_matrix()
#print(np.argwhere(data=='2016-10-08')[0][0])

res = getEvalData(data,'2015-10-15',predict_delta)

evaluation = np.array([res[0]])
#print(evaluation)
AvgPrevious(res[0])
#model.predict(evaluation)
#print(rawdata[0].loc['2013-04-28'])

NameError: name 'getEvalData' is not defined

In [29]:
# This function retrieves the previous 4 days given a single day for input from the .csv file.
def getEvalData(nparray,date,predict_delta):
    res = [[]]
    idx = np.argwhere(nparray==date)[0][0] - 5
    # if the given date does not have 5 previous values...
    if idx < 4 or idx + 7 > nparray.shape[0]:
        return
    else:
        for i in range(5):
            res[0].append(nparray[i+idx][1])
        predict_day = nparray[i+idx+predict_delta][1]
        predict_day_percent = (predict_day - res[0][4])/predict_day
        res.append(predict_day_percent)
        res.append(predict_day)
    return res
        

In [52]:
# Trying to convert the input data, which starts off as dataframeX, must take 5 days and then transpose it
# and then convert it into a numpy array

rawdata = pd.read_csv('data/bitcoin.csv',usecols=[8])
days = 5                                                      # Number of days we are looking at
predict_delta = 7                                             # Number of days we are predicting into the future 
number_of_items = rawdata.shape[0] - (predict_delta + days)   # number of tests
# size of our test 
test_size = number_of_items - 733

# five day blocks in a list
Xtrain = []    
# the values of the day we are trying to predict
Ytrain = []                                                   



# Splits the array of closing prices up into multiple arrays, each holding five days worth of prices
# then transposes each array and finally appends them onto one large matrix
for i in range(test_size - 300): 
    # finding the predict day
    predict = rawdata.head(days + i + predict_delta).tail(1).as_matrix().transpose().tolist()[0][0]
    last_day = rawdata.head(days + i).tail(1).as_matrix().transpose().tolist()[0][0]
    # Converting and adding data to arrays
    res = AvgPrevious(rawdata.head(days + i).tail(days).as_matrix().transpose().tolist()[0])
    Xtrain.append(res)
    Ytrain.append((predict - last_day)/predict)

# converts python arrays into numpy arrays
Xtrain = np.array(Xtrain)
Ytrain = np.array(Ytrain)

print(Xtrain)

[[133.13303107 143.61146811 137.96014388 115.80186426 104.09803346]
 [143.57832849 137.96014388 115.80186426 104.09803346  96.67368286]
 [138.15834532 115.80186426 104.09803346  96.67368286 111.63111111]
 ...
 [247.53603549 246.02392786 251.82282437 241.66842775 246.54947239]
 [246.00664251 251.82282437 241.66842775 246.54947239 243.20644665]
 [251.83991297 241.66842775 246.54947239 243.20644665 246.28233469]]


In [31]:
# number of evalutaion data points
eval_size = number_of_items - test_size

# evaluation input data
XEvaluation = []
# evaluation output data
YEvaluation = []

# appending evaluation data to appropriate arrays
for i in range(eval_size):
    # finding the day we want to predict a week out
    predict = rawdata.head(days + i + test_size + predict_delta).tail(1).as_matrix().transpose().tolist()[0][0]
    
    # finding the last day out of the 5 we are using as input to later calculate the percent difference 
    # of the change.  The percentage change is not the absolute value as we want to calculate negative percentages as well
    last_day = rawdata.head(days + i + test_size).tail(1).as_matrix().transpose().tolist()[0][0]
    
    #result after running the AvgPrevious function on the 5 days used as input
    res =rawdata.head(days + i + test_size).tail(days).as_matrix().transpose().tolist()[0]
    
    XEvaluation.append(res)
    YEvaluation.append((predict- last_day)/predict)
    
XEvaluation = np.array(XEvaluation)
YEvaluation = np.array(YEvaluation)

print(XEvaluation)
print(YEvaluation)

[[  387.49   402.97   391.73   392.15   394.97]
 [  402.97   391.73   392.15   394.97   380.29]
 [  391.73   392.15   394.97   380.29   379.47]
 ...
 [10931.4  10868.4  11359.4  11259.4  11171.4 ]
 [10868.4  11359.4  11259.4  11171.4  11440.7 ]
 [11359.4  11259.4  11171.4  11440.7  11786.3 ]]
[-6.76307609e-02  2.38712493e-02  1.83158712e-02 -4.59470944e-03
  2.08432903e-02  1.04431651e-03  4.20179241e-03  3.06563605e-02
 -2.61820097e-02 -5.95950658e-03  3.91466340e-02  7.51663679e-02
  6.68182613e-02  7.72043486e-02  8.32772867e-02  1.01143547e-01
  8.68128995e-02  1.03623387e-01  7.19462170e-02  8.58023986e-02
  3.14921329e-02  2.03082716e-02  5.11141471e-03  2.62871688e-02
 -1.07278276e-02 -1.22260669e-02 -1.14233493e-04  3.30483545e-02
 -2.26420434e-03 -6.85402585e-03 -5.16133742e-02 -7.97613401e-02
 -6.32557455e-02 -5.64298127e-02 -5.10906587e-02 -2.20074242e-02
 -1.08359504e-02  2.54926605e-02  2.68451484e-02  1.53359417e-02
  5.09076938e-03  6.86131037e-03  5.15575166e-03  8.2972

In [53]:
model = Sequential()

model.add(Dense(1,input_shape=(5,)))
model.add(Activation('softsign'))
model.add(Dense(1, activation='tanh'))
model.add(Dropout(5.0))

model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

H = model.fit(Xtrain, Ytrain, epochs=5)

Epoch 1/5
700/700 [==============================] - 1s 723us/step - loss: 0.0990 - acc: 0.0014
Epoch 2/5
700/700 [==============================] - 0s 144us/step - loss: 0.0289 - acc: 0.0014
Epoch 3/5
700/700 [==============================] - 0s 156us/step - loss: -0.0552 - acc: 0.0014
Epoch 4/5
700/700 [==============================] - 0s 134us/step - loss: -0.0655 - acc: 0.0014
Epoch 5/5
700/700 [==============================] - 0s 118us/step - loss: -0.0794 - acc: 0.0014


In [54]:
Test = [8070.8, 8891.21, 8516.24, 9477.84, 10016.49]
XVal = np.array([Test])
r = model.predict(XVal)
result = r[0][0]

if result >= 1 - 2/3:
    print("it will go up by 3%")
elif result > 0 - 2/3 and result < 1 - 2/3:
    print("it will stay between 3%")
else:
    print("it will go down by 3%")

print(result)

it will stay between 3%
0.013417426


In [55]:
score = model.evaluate(XEvaluation,YEvaluation,batch_size=100, verbose=1)
test_loss = score[0]
print("The evaluation loss is: " + str(test_loss))

733/733 [==============================] - 0s 223us/step
The evaluation loss is: 0.3282084818751555


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1)                 6         
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 2         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1)                 0         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


In [57]:
rawdata = pd.read_csv('data/bitcoin.csv',usecols=[3,8])
data = rawdata.as_matrix()
#print(np.argwhere(data=='2016-10-08')[0][0])

res = getEvalData(data,'2017-10-17',predict_delta)

evaluation = np.array([res[0]])
print(model.predict(evaluation))
print(res)

[[-0.10244986]]
[[5446.91, 5647.21, 5831.79, 5678.19, 5725.59], 0.03452258900025624, 5930.32]
